In [1]:
import pandas as pd

#1 Read keyword performance data (wa2_keyword_performance.xlsx)
performance_data_path = '/Users/baymaxshawn/Desktop/implementation_assessment/wa2_keyword_performance.xlsx'
performance_df = pd.read_excel(performance_data_path)

In [2]:
#2 Use the “date” field to determine the most recent date observed
end_date = performance_df['date'].max()

In [4]:
#3 Determine the date 30 days before the ending date range value
start_date = end_date - pd.DateOffset(days=30)
print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2022-03-06 00:00:00
End Date: 2022-04-05 00:00:00


In [36]:
# 4 Aggregate the metrics across the desired time period and calculate ACoS
aggregated_data = performance_df[(performance_df['date'] >= start_date) & (performance_df['date'] <= end_date)].groupby('keyword_id').agg({
    'impressions': 'sum',
    'clicks': 'sum',
    'cost': 'sum',
    'sales': 'sum',
    'units_ordered': 'sum'
}).reset_index()

# Calculate ACoS
aggregated_data['ACoS'] = aggregated_data['cost'] / aggregated_data['sales']

# Save unaggregated values 
unaggregated_values = performance_df[['keyword_id','keyword_text' ,'match_type', 'campaign_id', 'campaign_name', 'short_id', 'top_search_multiplier']].drop_duplicates()

# Merge aggregated data and unaggregated values
final_data = pd.merge(aggregated_data, unaggregated_values, on='keyword_id', how='left')
#print(final_data.head())


In [37]:
#5 Read the keyword bid file (wa2_keyword_bid.xlsx) and
bid_data_path = '/Users/baymaxshawn/Desktop/implementation_assessment/wa2_keyword_bid.xlsx'
bid_df = pd.read_excel(bid_data_path)
print("Column Names in bid_df:", bid_df.columns)

# map bid onto the aggregated performance table via keyword_id
final_data = pd.merge(final_data, bid_df[['keyword_id', 'bid', 'state']], on='keyword_id', how='left')
#print(final_data.head())

Column Names in bid_df: Index(['keyword_id', 'state', 'bid'], dtype='object')


In [38]:
#6 Given bid and top_search_multiplier, calculate effective_bid (effective_bid = bid * top_search_multiplier)
final_data['effective_bid'] = final_data['bid'] * final_data['top_search_multiplier']


In [39]:
# Read Product Database
product_database_path = '/Users/baymaxshawn/Desktop/implementation_assessment/wa2_product_database.xlsx'  
product_database = pd.read_excel(product_database_path)
#print("Column Names in product_database:", product_database.columns)

# merge breakeven to data
final_data = pd.merge(final_data, product_database[['short_id', 'Breakeven ACoS', 'Breakeven Cost']], on='short_id', how='left')
#print(final_data.head())

In [40]:
#7 Apply bid optimization rules(1-5)
# Rule 1
rule1_condition = final_data['ACoS'] > (final_data['Breakeven ACoS'] + 0.10)
final_data.loc[rule1_condition, 'effective_bid'] *= 0.75

# Rule 2
rule2_condition = (final_data['ACoS'] < (final_data['Breakeven ACoS'] - 0.05)) & (final_data['units_ordered'] >= 2)
final_data.loc[rule2_condition, 'effective_bid'] *= 1.15

# Rule 3
rule3_condition = (final_data['sales'] == 0) & (final_data['cost'] > 1.25 * final_data['Breakeven Cost'])
final_data.loc[rule3_condition, 'effective_bid'] *= 0.75

# Rule 4
rule4_condition = final_data['ACoS'] > (2.75 * final_data['Breakeven ACoS'])
final_data.loc[rule4_condition, 'Final Proposed State'] = 'paused'

# Rule 5
rule5_condition = (final_data['sales'] == 0) & (final_data['cost'] > 2.5 * final_data['Breakeven Cost'])
final_data.loc[rule5_condition, 'Final Proposed State'] = 'paused'

In [41]:
final_data['conversions'] = final_data['sales'] 
final_data['Breakeven ACoS+10'] = final_data['Breakeven ACoS'] + 0.10
final_data['Breakeven ACoS-5'] = final_data['Breakeven ACoS'] - 0.05
final_data['Breakeven ACoSx2.75'] = final_data['Breakeven ACoS'] * 2.75
final_data['Breakeven Costx1.25'] = final_data['Breakeven Cost'] * 1.25
final_data['Breakeven Costx2.5'] = final_data['Breakeven Cost'] * 2.5
final_data['Bid Optimization Rule(s) Applied'] = ''  
final_data['Final Proposed Effective Bid'] = 0.0  

In [42]:
#8 For Final Proposed Bid
final_data['Final Proposed Bid'] = final_data['effective_bid'] / final_data['top_search_multiplier']

In [43]:
#9 Note the bid optimization rule applied/type of change
final_data.loc[rule1_condition, 'Bid Optimization Rule(s) Applied'] = 'Rule 1: Decrease by 25%'
final_data.loc[rule2_condition, 'Bid Optimization Rule(s) Applied'] = 'Rule 2: Increase by 15%'
final_data.loc[rule3_condition, 'Bid Optimization Rule(s) Applied'] = 'Rule 3: Decrease by 25%'
final_data.loc[rule4_condition, 'Bid Optimization Rule(s) Applied'] = 'Rule 4: Paused'
final_data.loc[rule5_condition, 'Bid Optimization Rule(s) Applied'] = 'Rule 5: Paused'
final_data.loc[:, 'Final Proposed Effective Bid'] = final_data['effective_bid'] 

In [46]:
#10 Generate output file
final_data['Final Proposed State'] = 'Active' 
final_data.loc[rule4_condition | rule5_condition, 'Final Proposed State'] = 'Paused'

selected_columns = [
    'keyword_id',
    'keyword_text',
    'campaign_name',
    'short_id',
    'bid',
    'top_search_multiplier',
    'effective_bid',
    'impressions',
    'clicks',
    'cost',
    'sales',
    'ACoS',
    'units_ordered',
    'conversions',
    'Breakeven ACoS',
    'Breakeven ACoS+10',
    'Breakeven ACoS-5',
    'Breakeven ACoSx2.75',
    'Breakeven Cost',
    'Breakeven Costx1.25',
    'Breakeven Costx2.5',
    'Bid Optimization Rule(s) Applied',
    'Final Proposed Effective Bid',
    'Final Proposed Bid',
    'Final Proposed State'
]

final_data = final_data[selected_columns]

output_path = '/Users/baymaxshawn/Desktop/implementation_assessment/output_file.xlsx'  
final_data.to_excel(output_path, index=False)

#print(final_data.head())
print("Column Names in final_data:", final_data.columns)

Column Names in final_data: Index(['keyword_id', 'keyword_text', 'campaign_name', 'short_id', 'bid',
       'top_search_multiplier', 'effective_bid', 'impressions', 'clicks',
       'cost', 'sales', 'ACoS', 'units_ordered', 'conversions',
       'Breakeven ACoS', 'Breakeven ACoS+10', 'Breakeven ACoS-5',
       'Breakeven ACoSx2.75', 'Breakeven Cost', 'Breakeven Costx1.25',
       'Breakeven Costx2.5', 'Bid Optimization Rule(s) Applied',
       'Final Proposed Effective Bid', 'Final Proposed Bid',
       'Final Proposed State'],
      dtype='object')
